# Real world example: Maintenance scripts

In [1]:
from pybis import Openbis
url    = 'https://main.datastore.bam.de/'
pat    = '' # leave empty to read token from file or authenticate with username
userid = '' # leave empty to use the username from your local machine
space  = '' # leave empty to use the users home space
try:
    from os import environ
    pat = pat or open(environ.get('OPENBIS_PAT_FILE', 'OPENBIS_PAT.txt'), 'r').read().strip()
    o = Openbis(url, token=pat)
    userid = o.token.split('-')[1]
except:
    from getpass import getuser, getpass
    o = Openbis(url)
    userid = userid.lower() or getuser()
    password = getpass('Enter password for user {} at {}: '.format(userid, url))
    o.login(userid, password)
server_info = o.get_server_information()
person = o.get_person(userid)
space = space.upper() or person.space

print('Server: {} (openBIS {}, API {})'.format(o.hostname, server_info.openbis_version, server_info.api_version))
print('UserId: {} ({} {}, {})'.format(person.userId, person.firstName, person.lastName, person.email))
print('Space : {}'.format(space))

Server: main.datastore.bam.de (openBIS 20.10.11.1, API 3.7)
UserId: cmadaria (Carlos Madariaga, Carlos.Madariaga@bam.de)
Space : VP.1_CMADARIA


## Scenario
In your OpenBIS instance exists a number of objects of type CHEMICAL. We create maintenance scripts to perform the following actions:
* Check properties: checks the expiration date and saves a list of expired chemicals in a custom format.
* Storage change: changes the location of some chemicals to another room.

This example shows the **interactive development process** - step by step from the first line to the complete scripts.

**As all users work on the same instance, we will not modify the global INVENTORY but limit the actions to the user's space. But this code may be used for the inventory as well, just change the SPACE limitation condition.**

## OPTIONAL: create example data for this example
This example needs some objects of type CHEMICAL to work. If you don't have example objects to play with you may create some with the following code.

In [2]:
PROJECT='PYBISTUTORIAL'
COLLECTION='MYCHEMICALS'

my_space = o.get_space(space)
try:
    proj = my_space.get_project(PROJECT)
except ValueError:
    proj=o.new_project(space=space, code=PROJECT, description='just for learning pyBIS')
    proj.save()
try:
    coll = my_space.get_collection(COLLECTION)
except ValueError:
    coll=o.new_collection(project=proj, code=COLLECTION, type='COLLECTION')
    coll.save()
c = o.new_object(type='CHEMICAL', space=space, collection=coll, props={
    '$name': 'Water',
    'description': 'just pure water',
    'substance_empty': False,
    'manufacturer': 'ACME Corp.',
    'hazardous_substance': False,
    'bam_location_complete': 'UE_02_2_310',
    'date_expiration': '2023-12-31',
    'bam_oe': 'OE_1'
})
c.save()
c = o.new_object(type='CHEMICAL', space=space, collection=coll, props={
    '$name': 'Ethanol',
    'description': 'hicks!',
    'substance_empty': False,
    'manufacturer': 'ACME Corp.',
    'hazardous_substance': True,
    'bam_location_complete': 'UE_02_2_310',
    'date_expiration': '2025-12-31',
    'bam_oe': 'OE_1'
})
c.save()
c = o.new_object(type='CHEMICAL', space=space, collection=coll, props={
    '$name': 'Propanol',
    'description': 'bäh!',
    'substance_empty': False,
    'manufacturer': 'ACME Corp.',
    'hazardous_substance': True,
    'bam_location_complete': 'UE_02_2_309',
    'date_expiration': '2024-12-31',
    'bam_oe': 'OE_1'
})
c.save()

experiment successfully created.
sample successfully created.
sample successfully created.
sample successfully created.


## Example: Check for expiry

Check for expired CHEMICALs and export the list to a CSV file in a custom format.

### Searching all objects of type CHEMICAL

In [3]:
o.get_objects(type='CHEMICAL')

,permId,identifier,registrationDate,modificationDate,type,registrator,modifier
0,20251030134504209-20086,/VP.1_CMADARIA/PYBISTUTORIAL/CHEM8049,2025-10-30 13:45:04,2025-10-30 13:45:04,CHEMICAL,cmadaria,cmadaria
1,20251030134505688-20087,/VP.1_CMADARIA/PYBISTUTORIAL/CHEM8050,2025-10-30 13:45:06,2025-10-30 13:45:06,CHEMICAL,cmadaria,cmadaria
2,20251030134507162-20088,/VP.1_CMADARIA/PYBISTUTORIAL/CHEM8051,2025-10-30 13:45:07,2025-10-30 13:45:07,CHEMICAL,cmadaria,cmadaria
3,20251017113151333-19165,/BAM_MATERIALS/2.4_MATERIALS_OPEN/CHEM7542,2025-10-17 11:31:51,2025-10-17 13:46:49,CHEMICAL,pschulte,aagasty
4,20250310160631499-9733,/BAM_MATERIALS/VP.1_MATERIALS_OPEN/CHEM5403,2025-03-10 16:06:31,2025-03-10 16:06:32,CHEMICAL,cdemidov,cdemidov
5,20250311115442981-9738,/VP.1_MATERIALS/VP1_MATERIALS_DEMO/CHEM5405,2025-03-11 11:54:43,2025-03-11 11:54:44,CHEMICAL,aschelle,aschelle
6,20251017125831675-19190,/VP.1_JPIZARRO/TEST_PROJECT/CHEM7586,2025-10-17 12:58:32,2025-10-17 12:58:32,CHEMICAL,jpizarro,jpizarro
7,20251017125916007-19191,/VP.1_JPIZARRO/TEST_PROJECT/CHEM7587,2025-10-17 12:59:16,2025-10-17 12:59:16,CHEMICAL,jpizarro,jpizarro
8,20251017130151711-19193,/VP.1_JPIZARRO/TEST_PROJECT/CHEM7588,2025-10-17 13:01:52,2025-10-17 13:01:52,CHEMICAL,jpizarro,jpizarro
9,20241212155644270-8972,/VP.1_CDEMIDOV/PYBISTUTORIAL/CHEM5252,2024-12-12 15:56:44,2024-12-12 15:56:44,CHEMICAL,cdemidov,cdemidov


### Limiting search to a SPACE

In [4]:
o.get_objects(type='CHEMICAL', space=space.code)

,permId,identifier,registrationDate,modificationDate,type,registrator,modifier
0,20251030134504209-20086,/VP.1_CMADARIA/PYBISTUTORIAL/CHEM8049,2025-10-30 13:45:04,2025-10-30 13:45:04,CHEMICAL,cmadaria,cmadaria
1,20251030134505688-20087,/VP.1_CMADARIA/PYBISTUTORIAL/CHEM8050,2025-10-30 13:45:06,2025-10-30 13:45:06,CHEMICAL,cmadaria,cmadaria
2,20251030134507162-20088,/VP.1_CMADARIA/PYBISTUTORIAL/CHEM8051,2025-10-30 13:45:07,2025-10-30 13:45:07,CHEMICAL,cmadaria,cmadaria


### Include needed properties in result

In [5]:
o.get_objects(type='CHEMICAL', space=space.code, props=['$name', 'date_expiration'])

,permId,identifier,registrationDate,modificationDate,type,registrator,modifier,$NAME,DATE_EXPIRATION
0,20251030134504209-20086,/VP.1_CMADARIA/PYBISTUTORIAL/CHEM8049,2025-10-30 13:45:04,2025-10-30 13:45:04,CHEMICAL,cmadaria,cmadaria,Water,2023-12-31
1,20251030134505688-20087,/VP.1_CMADARIA/PYBISTUTORIAL/CHEM8050,2025-10-30 13:45:06,2025-10-30 13:45:06,CHEMICAL,cmadaria,cmadaria,Ethanol,2025-12-31
2,20251030134507162-20088,/VP.1_CMADARIA/PYBISTUTORIAL/CHEM8051,2025-10-30 13:45:07,2025-10-30 13:45:07,CHEMICAL,cmadaria,cmadaria,Propanol,2024-12-31


### Check for expiration date

In [7]:
ac = o.get_objects(type='CHEMICAL', space=space.code, props=['$name', 'date_expiration'])
today = '2025-11-27'
expired = []
for c in ac:
    if c.props['date_expiration'] < today:
        print(c.code)
        expired.append(c)

CHEM8049
CHEM8051


You will run into trouble if there are chemicals without the expiration date (which is not mandatory). A value of `None` can't be compared to a string!

In [8]:
expired = []
for c in ac:
    if c.props['date_expiration'] is not None and c.props['date_expiration'] < today:
        print(c.code)
        expired.append(c)

CHEM8049
CHEM8051


**Hint:** getting a value twice with `[]` may lead to performance issues. There are several workarounds:
```python
ed = c.props['date_expiration']
if ed is not None and ed < today:
    ...
```
or:
```python
if (c.props['date_expiration'] or '9999-12-31') < today:
    ...
```
The explicit `for`-loop can also be replaced with a shorter list comprehension, which lead to very compact code:

In [10]:
expired = [c for c in ac if (c.props['date_expiration'] or '9999-12-31') < today]
expired

[attribute            value
 -------------------  ----------------------------------------
 code                 CHEM8049
 permId               20251030134504209-20086
 identifier           /VP.1_CMADARIA/PYBISTUTORIAL/CHEM8049
 type                 CHEMICAL
 project              /VP.1_CMADARIA/PYBISTUTORIAL
 parents              --NOT FETCHED--
 children             --NOT FETCHED--
 components           []
 space                VP.1_CMADARIA
 experiment           /VP.1_CMADARIA/PYBISTUTORIAL/MYCHEMICALS
 tags                 []
 registrator          cmadaria
 registrationDate     2025-10-30 13:45:04
 modifier             cmadaria
 modificationDate     2025-10-30 13:45:04
 container
 frozen               False
 frozenForComponents  False
 frozenForChildren    False
 frozenForParents     False
 frozenForDataSets    False,
 attribute            value
 -------------------  ----------------------------------------
 code                 CHEM8051
 permId               20251030134507162-20088

### Use current date for comparison instead of hardcoded
A script should not contain a manually entered date - just get the current date.

In [11]:
from datetime import date
today = date.today().isoformat()
expired = [c for c in ac if c.props['date_expiration'] is not None and c.props['date_expiration'] < today]
for c in expired:
    print(c.code)

CHEM8049
CHEM8051


### Save this list in a CSV file
Now the list shoul be saved in a file that can be given to the colleagues maintaining the chemicals. A custom column layout should be used.

In [12]:
import csv
fields = ('$name', 'hazardous_substance', 'description', 'bam_location_complete', 'date_expiration')
with open('EXPIRED_CHEMICALS.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)
    for c in expired:
        csvwriter.writerow([c.props[f] for f in fields])

### Putting it all together - the complete script
This is a combination of the above code in one script. All import statements were moved to the top.

In [14]:
from pybis import Openbis
from datetime import date
import csv

fields = ('$name', 'hazardous_substance', 'description', 'bam_location_complete', 'date_expiration')

ac = o.get_objects(type='CHEMICAL', space=space.code, props=fields)

# filter out expired chemicals
from datetime import date
today = date.today().isoformat()
expired = [c for c in ac if (c.props['date_expiration'] or '9999-12-31') < today]

print(expired)

# write list as csv file
fields = ('$name', 'hazardous_substance', 'description', 'bam_location_complete', 'date_expiration')
with open('EXPIRED_CHEMICALS.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)
    for c in expired:
        csvwriter.writerow([c.props[f] for f in fields])

o.logout()

[attribute            value
-------------------  ----------------------------------------
code                 CHEM8049
permId               20251030134504209-20086
identifier           /VP.1_CMADARIA/PYBISTUTORIAL/CHEM8049
type                 CHEMICAL
project              /VP.1_CMADARIA/PYBISTUTORIAL
parents              --NOT FETCHED--
children             --NOT FETCHED--
components           []
space                VP.1_CMADARIA
experiment           /VP.1_CMADARIA/PYBISTUTORIAL/MYCHEMICALS
tags                 []
registrator          cmadaria
registrationDate     2025-10-30 13:45:04
modifier             cmadaria
modificationDate     2025-10-30 13:45:04
container
frozen               False
frozenForComponents  False
frozenForChildren    False
frozenForParents     False
frozenForDataSets    False, attribute            value
-------------------  ----------------------------------------
code                 CHEM8051
permId               20251030134507162-20088
identifier           /VP.

## Example: Room number change
A storage for chemicals has moved from one room to another. The location of all CHEMICALs with the old room number need to be changed to a new room number.

### Get rooms from BAM_LOCATION_COMPLETE

To change the room, we do not just need the label of the vocabulary term, but the CODE (internal name) of the entry. We could also get this via pyBIS, but this is not usable in current pyBIS versions. Please use the Vocabulare Browser in the ELN to get the codes.

In [15]:
OLD_ROOMCODE = 'UE_02_2_310' # 'UE/02/2/310'
NEW_ROOMCODE = 'UE_02_2_309' # 'UE/02/2/309'

### Get list of CHEMICALs with old room number
This is almost the same code as in the previous example. The only new thing is the `where=` statement, which returns just chemicals with the property `bam_location_complete` matching the room.

In [16]:
cc = o.get_objects(type='CHEMICAL', space=space.code, props=['$name', 'bam_location_complete'], where={'bam_location_complete': OLD_ROOMCODE})
cc

,permId,identifier,registrationDate,modificationDate,type,registrator,modifier,$NAME,BAM_LOCATION_COMPLETE
0,20251030134504209-20086,/VP.1_CMADARIA/PYBISTUTORIAL/CHEM8049,2025-10-30 13:45:04,2025-10-30 13:45:04,CHEMICAL,cmadaria,cmadaria,Water,UE_02_2_310
1,20251030134505688-20087,/VP.1_CMADARIA/PYBISTUTORIAL/CHEM8050,2025-10-30 13:45:06,2025-10-30 13:45:06,CHEMICAL,cmadaria,cmadaria,Ethanol,UE_02_2_310


### Iterate over list and set new room number

Change every single object in list and save it. This works well for a small number of objects.

In [19]:
for c in cc:
    c.props['bam_location_complete'] = NEW_ROOMCODE
    c.save()
    print(c.props['bam_location_complete'])

sample successfully updated.
UE_02_2_309
sample successfully updated.
UE_02_2_309


### Like before, but with transactions

In [20]:
trans = o.new_transaction()
for c in cc:
    c.props['bam_location_complete'] = NEW_ROOMCODE
    trans.add(c)
trans.commit()

2 sample(s) updated.


### Putting it all together - the complete script
This is a combination of the above code in one script. Do not forget to adjust the room numbers every time you run it.

In [21]:
from pybis import Openbis

OLD_ROOMCODE = 'UE_02_2_310' # 'UE/02/2/310'
NEW_ROOMCODE = 'UE_02_2_309' # 'UE/02/2/305'

# get all chemicals in space that have the old location
cc = o.get_objects(type='CHEMICAL', space=space.code, props=['$name', 'bam_location_complete'], where={'bam_location_complete': OLD_ROOMCODE})

# create and execute transaction to change objects to new location in just one action on the server
trans = o.new_transaction()
for c in cc:
    c.props['bam_location_complete'] = NEW_ROOMCODE
    trans.add(c)
trans.commit()

cc = o.get_objects(type='CHEMICAL', space=space.code, props=['$name', 'bam_location_complete'])
print(cc)

o.logout()

    permId                   identifier                             registrationDate     modificationDate     type      registrator    modifier    $NAME     BAM_LOCATION_COMPLETE
--  -----------------------  -------------------------------------  -------------------  -------------------  --------  -------------  ----------  --------  -----------------------
 0  20251030134504209-20086  /VP.1_CMADARIA/PYBISTUTORIAL/CHEM8049  2025-10-30 13:45:04  2025-10-30 14:06:06  CHEMICAL  cmadaria       cmadaria    Water     UE_02_2_309
 1  20251030134505688-20087  /VP.1_CMADARIA/PYBISTUTORIAL/CHEM8050  2025-10-30 13:45:06  2025-10-30 14:06:06  CHEMICAL  cmadaria       cmadaria    Ethanol   UE_02_2_309
 2  20251030134507162-20088  /VP.1_CMADARIA/PYBISTUTORIAL/CHEM8051  2025-10-30 13:45:07  2025-10-30 13:45:07  CHEMICAL  cmadaria       cmadaria    Propanol  UE_02_2_309


## Logout

In [22]:
o.logout()